In [4]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import torch.optim as optim
import timeit
import ntpath
import tqdm


torch.cuda.set_device(0)

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
base_dir = os.path.dirname("antrenare/")
personaje = ["barney", "betty", "fred", "wilma"]
im_height = 360
im_width = 480

cuda:1


In [2]:
base_dir = os.path.dirname("antrenare/")
personaje = ["barney", "betty", "fred", "wilma"]
im_height = 360
im_width = 480
nr = 0
dataset_dir = os.path.dirname("dataset/images/train/")
for pers in personaje:
    img_dir = os.path.join(base_dir, pers)
    for img in tqdm.tqdm(os.listdir(img_dir)):
        nr += 1
        current_img = cv.imread(os.path.join(img_dir, img))
        cv.imwrite(os.path.join(dataset_dir, (str(nr)).zfill(5) + ".jpg"), current_img)


100%|██████████| 1000/1000 [00:08<00:00, 123.00it/s]


In [4]:
nr = 0
dataset_ann = os.path.dirname("dataset/annotations/train/")
# Delete old annotations
for file in os.listdir(dataset_ann):
    os.remove(os.path.join(dataset_ann, file))
for pers in personaje:
    ann_dir = os.path.join(base_dir, pers + "_annotations.txt")
    ann_file = open(ann_dir, "r")
    last_file_name = ""
    for line in ann_file:
        line = line.split()
        file_name = line[0]
        if file_name != last_file_name:
            nr += 1
        x_min = line[1]
        y_min = line[2]
        x_max = line[3]
        y_max = line[4]


        # Transforma in formatul yolo

        x_center = int(x_min) + (int(x_max) - int(x_min)) / 2
        y_center = int(y_min) + (int(y_max) - int(y_min)) / 2
        width = int(x_max) - int(x_min)
        height = int(y_max) - int(y_min)

        x_center = x_center / im_width
        y_center = y_center / im_height
        width = width / im_width
        height = height / im_height
        last_file_name = file_name

        
        with open(os.path.join(dataset_ann, (str(nr)).zfill(5) + ".txt"), "a") as f:
            f.write("0 " + str(round(x_center,3)) + " " + str(round(y_center,3)) + " " + str(round(width, 3)) + " " + str(round(height,3)) + "\n")



In [8]:
base_dir = os.path.dirname("")
# personaje = ["barney", "betty", "fred", "wilma"]
im_height = 360
im_width = 480
nr = 0
dataset_dir = os.path.dirname("dataset/images/validation/")
img_dir = os.path.join(base_dir, "validare")
for img in tqdm.tqdm(os.listdir(img_dir)):
    nr += 1
    current_img = cv.imread(os.path.join(img_dir, img))
    cv.imwrite(os.path.join(dataset_dir, (str(nr)).zfill(5) + ".jpg"), current_img)


100%|██████████| 200/200 [00:01<00:00, 159.05it/s]


In [9]:
nr = 0
dataset_ann = os.path.dirname("dataset/annotations/validation/")
for file in os.listdir(dataset_ann):
    os.remove(os.path.join(dataset_ann, file))

ann_file = open(os.path.join(base_dir, "task1_gt_validare.txt"), "r")
for line in ann_file:
    line = line.split()
    file_name = line[0]
    if file_name != last_file_name:
        nr += 1
    x_min = line[1]
    y_min = line[2]
    x_max = line[3]
    y_max = line[4]

    # centrul imaginii
    x_center = int(x_min) + (int(x_max) - int(x_min)) / 2  
    y_center = int(y_min) + (int(y_max) - int(y_min)) / 2
    # latimea si inaltimea se obtin prin diferenta dintre coordonata maxima si cea minima
    width = int(x_max) - int(x_min)
    height = int(y_max) - int(y_min)


    # raportam totul la dimensiunea imaginii
    x_center = x_center / im_width
    y_center = y_center / im_height
    width = width / im_width
    height = height / im_height
    last_file_name = file_name

    with open(os.path.join(dataset_ann, (str(nr)).zfill(5) + ".txt"), "a") as f:
        f.write("0 " + str(round(x_center,3)) + " " + str(round(y_center,3)) + " " + str(round(width, 3)) + " " + str(round(height,3)) + "\n")
            







In [5]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.to('cuda')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [6]:
results = model.train(
    data='soneage.yaml',
    epochs=5,
    imgsz=640,
    batch=32,
    workers=0,
    )

engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=soneage.yaml, epochs=5, time=None, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=0, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False

train: Scanning C:\Users\gheto\Desktop\yolo\dataset\labels\train.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]
val: Scanning C:\Users\gheto\Desktop\yolo\dataset\labels\validation.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]

Plotting labels to runs\detect\train2\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      4.32G      1.121        1.4       1.14         99        640: 100%|██████████| 125/125 [01:14<00:00,  1.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.90it/s]

...

        5/5      4.34G     0.9485     0.5012      1.047        129        640: 100%|██████████| 125/125 [01:28<00:00,  1.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.82it/s]

                   all        200        322      0.996      0.994      0.995      0.802



5 epochs completed in 0.123 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.0.231 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.34it/s]


                   all        200        322      0.996      0.994      0.995      0.802
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train2


In [ ]:
print(results)

In [7]:
for im_name in os.listdir("validare"):
    rez = model(["validare/" + im_name])
    for rezu in rez:
        # print(rezu.boxes)
        bbox = rezu.boxes.xyxy
        # Show image with detections
        img = cv.imread("validare/" + im_name)
        for i in range(len(bbox)):
            cv.rectangle(img, (int(bbox[i][0]), int(bbox[i][1])), (int(bbox[i][2]), int(bbox[i][3])), (0, 0, 255), 2)
        cv.imshow("img", img)
        cv.waitKey(0)
        cv.destroyAllWindows()    


0: 480x640 1 face, 75.0ms
Speed: 3.0ms preprocess, 75.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 faces, 28.0ms
Speed: 3.0ms preprocess, 28.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 27.0ms
Speed: 3.0ms preprocess, 27.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 21.0ms
Speed: 3.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 22.0ms
Speed: 2.2ms preprocess, 22.0ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 faces, 22.0ms
Speed: 3.0ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 30.0ms
Speed: 2.0ms preprocess, 30.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 21.5ms
Speed: 2.0ms preprocess, 21.5ms inference, 11.0ms postprocess per image at shape (1, 3, 480, 640)

0: 4

In [8]:
def intersection_over_union(bbox_a, bbox_b):
        x_a = max(bbox_a[0], bbox_b[0])
        y_a = max(bbox_a[1], bbox_b[1])
        x_b = min(bbox_a[2], bbox_b[2])
        y_b = min(bbox_a[3], bbox_b[3])

        inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

        box_a_area = (bbox_a[2] - bbox_a[0] + 1) * (bbox_a[3] - bbox_a[1] + 1)
        box_b_area = (bbox_b[2] - bbox_b[0] + 1) * (bbox_b[3] - bbox_b[1] + 1)

        if(box_a_area + box_b_area - inter_area == 0):
            return 0
        iou = inter_area / float(box_a_area + box_b_area - inter_area)

        return iou
def compute_average_precision(rec, prec):
        # functie adaptata din 2010 Pascal VOC development kit
        m_rec = np.concatenate(([0], rec, [1]))
        m_pre = np.concatenate(([0], prec, [0]))
        for i in range(len(m_pre) - 1, -1, 1):
            m_pre[i] = max(m_pre[i], m_pre[i + 1])
        m_rec = np.array(m_rec)
        i = np.where(m_rec[1:] != m_rec[:-1])[0] + 1
        average_precision = np.sum((m_rec[i] - m_rec[i - 1]) * m_pre[i])
        return average_precision

def eval_detections(detections, file_names):
        ground_truth_file = np.loadtxt("task1_gt_validare.txt", dtype='str')
        ground_truth_file_names = np.array(ground_truth_file[:, 0])
        ground_truth_detections = np.array(ground_truth_file[:, 1:], np.int64)
        print(ground_truth_detections)
        num_gt_detections = len(ground_truth_detections)  # numar total de adevarat pozitive
        gt_exists_detection = np.zeros(num_gt_detections)


        num_detections = len(detections)
        true_positive = np.zeros(num_detections)
        false_positive = np.zeros(num_detections)
        duplicated_detections = np.zeros(num_detections)

        for detection_idx in range(num_detections):
            
            indices_detections_on_image = np.where(ground_truth_file_names == file_names[detection_idx])[0]
            print(ground_truth_file_names)
            print(file_names[detection_idx])
            gt_detections_on_image = ground_truth_detections[indices_detections_on_image]
            bbox = detections[detection_idx]
            max_overlap = -1
            index_max_overlap_bbox = -1
            for gt_idx, gt_bbox in enumerate(gt_detections_on_image):
                overlap = intersection_over_union(bbox, gt_bbox)
                if overlap > max_overlap:
                    max_overlap = overlap
                    index_max_overlap_bbox = indices_detections_on_image[gt_idx]

            # clasifica o detectie ca fiind adevarat pozitiva / fals pozitiva
            if max_overlap >= 0.3:
                if gt_exists_detection[index_max_overlap_bbox] == 0:
                    true_positive[detection_idx] = 1
                    gt_exists_detection[index_max_overlap_bbox] = 1
                else:
                    false_positive[detection_idx] = 1
                    duplicated_detections[detection_idx] = 1
            else:
                false_positive[detection_idx] = 1

        cum_false_positive = np.cumsum(false_positive)
        cum_true_positive = np.cumsum(true_positive)
        rec = cum_true_positive / num_gt_detections
        prec = cum_true_positive / (cum_true_positive + cum_false_positive)
        average_precision = compute_average_precision(rec, prec)
        plt.plot(rec, prec, '-')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Average precision %.3f' % average_precision)
        plt.savefig(os.path.join("precizie_medie.png"))
        plt.show()

In [9]:
fisiere_det = ["validare/" + im_name for im_name in os.listdir("validare")]
detect = model(fisiere_det)
detections = []
file_names = []
nr = 0
for det in detect:
    bbox = det.boxes.xyxy
    nr += 1
    for i in range(len(bbox)):
        detections.append([int(bbox[i][0]), int(bbox[i][1]), int(bbox[i][2]), int(bbox[i][3])])
        file_names.append(str(nr).zfill(4) + ".jpg")

scores = []
for i in range(len(detections)):
    scores.append(1)

np.save(f"rezultate/detections_all_faces.npy", detections)
np.save(f"rezultate/file_names_all_faces.npy", file_names)
np.save(f"rezultate/scores_all_faces.npy", scores)

# detections = np.array(detections, np.int64)
# print(detections)
# print(file_names)
eval_detections(detections,  file_names)


0: 480x640 1 face, 1: 480x640 2 faces, 2: 480x640 1 face, 3: 480x640 1 face, 4: 480x640 1 face, 5: 480x640 2 faces, 6: 480x640 1 face, 7: 480x640 1 face, 8: 480x640 1 face, 9: 480x640 3 faces, 10: 480x640 1 face, 11: 480x640 1 face, 12: 480x640 3 faces, 13: 480x640 1 face, 14: 480x640 3 faces, 15: 480x640 1 face, 16: 480x640 2 faces, 17: 480x640 1 face, 18: 480x640 1 face, 19: 480x640 1 face, 20: 480x640 1 face, 21: 480x640 1 face, 22: 480x640 1 face, 23: 480x640 3 faces, 24: 480x640 2 faces, 25: 480x640 2 faces, 26: 480x640 2 faces, 27: 480x640 2 faces, 28: 480x640 1 face, 29: 480x640 2 faces, 30: 480x640 1 face, 31: 480x640 2 faces, 32: 480x640 1 face, 33: 480x640 1 face, 34: 480x640 2 faces, 35: 480x640 2 faces, 36: 480x640 1 face, 37: 480x640 1 face, 38: 480x640 4 faces, 39: 480x640 1 face, 40: 480x640 1 face, 41: 480x640 1 face, 42: 480x640 1 face, 43: 480x640 3 faces, 44: 480x640 3 faces, 45: 480x640 3 faces, 46: 480x640 2 faces, 47: 480x640 5 faces, 48: 480x640 2 faces, 49: 480

<Figure size 640x480 with 1 Axes>